In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt

DATA_PATH = "../data"
H_SETTING = "hadamard_FISTA_p-5_lmd-1_m-128"

def interpolate_sparse_matrices(A_t, A_t1, alpha=1.0, beta=1.0):
    """
    中間時刻 t + 0.5 のスパース行列を補完する関数。
    
    Parameters:
    - A_t: scipy.sparse.csr_matrix
        時刻 t のスパース行列
    - A_t1: scipy.sparse.csr_matrix
        時刻 t+1 のスパース行列
    - alpha: float
        位置の距離に対する重み
    - beta: float
        値の差に対する重み
    
    Returns:
    - A_t_half: scipy.sparse.csr_matrix
        中間時刻 t + 0.5 のスパース行列
    """
    # スパース行列から非ゼロ要素を抽出
    def get_nonzero_elements(A):
        A_coo = A.tocoo()
        return list(zip(A_coo.row, A_coo.col, A_coo.data))
    
    S_t = get_nonzero_elements(A_t)
    S_t1 = get_nonzero_elements(A_t1)
    
    n_t = len(S_t)
    n_t1 = len(S_t1)
    
    if n_t == 0 and n_t1 == 0:
        return csr_matrix(A_t.shape)
    
    # コスト行列の構築
    cost_matrix = np.zeros((n_t, n_t1))
    for i, (r1, c1, v1) in enumerate(S_t):
        for j, (r2, c2, v2) in enumerate(S_t1):
            distance = np.sqrt((r1 - r2)**2 + (c1 - c2)**2)
            value_diff = abs(v1 - v2)
            cost_matrix[i, j] = alpha * distance + beta * value_diff
    
    # ハンガリアン法による最適マッチング
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # マッチングの閾値を設定（必要に応じて）
    # ここでは全てのマッチングを有効とする
    matched_t = set()
    matched_t1 = set()
    for i, j in zip(row_ind, col_ind):
        # オプション: コストが一定以下の場合のみマッチングとする
        # threshold = some_value
        # if cost_matrix[i, j] <= threshold:
        #     matched_t.add(i)
        #     matched_t1.add(j)
        # else:
        #     continue
        matched_t.add(i)
        matched_t1.add(j)
    
    # 補完された非ゼロ要素のリスト
    interpolated_elements = []
    
    # マッチングされた要素の補完
    for i, j in zip(row_ind, col_ind):
        r1, c1, v1 = S_t[i]
        r2, c2, v2 = S_t1[j]
        r_half = (r1 + r2) / 2
        c_half = (c1 + c2) / 2
        v_half = (v1 + v2) / 2
        # 位置を最も近い整数に丸める
        r_half_int = int(round(r_half))
        c_half_int = int(round(c_half))
        interpolated_elements.append((r_half_int, c_half_int, v_half))
    
    # マッチングされなかった要素の処理
    # A(t) の未マッチ要素は減少
    for i, (r, c, v) in enumerate(S_t):
        if i not in matched_t:
            # 値を半減
            v_half = v / 2
            if v_half != 0:
                interpolated_elements.append((r, c, v_half))
    
    # A(t+1) の未マッチ要素は出現
    for j, (r, c, v) in enumerate(S_t1):
        if j not in matched_t1:
            # 値を半増
            v_half = v / 2
            interpolated_elements.append((r, c, v_half))
    
    # 重複する位置の値を合計
    from collections import defaultdict
    position_dict = defaultdict(float)
    for r, c, v in interpolated_elements:
        position_dict[(r, c)] += v
    
    # 最終的な非ゼロ要素をリスト化
    final_elements = [(r, c, v) for (r, c), v in position_dict.items() if v != 0]
    
    # 行列の構築
    if final_elements:
        rows, cols, data = zip(*final_elements)
        A_t_half = csr_matrix((data, (rows, cols)), shape=A_t.shape)
    else:
        A_t_half = csr_matrix(A_t.shape)
    
    return A_t_half


def matrix_to_tensor(H, m, n):
    H_tensor = np.zeros((m, m, n, n))
    for i in range(m):
        for j in range(m):
            for k in range(n):
                for l in range(n):
                    H_tensor[i, j, k, l] = H[i * m + j, k * n + l]
    return H_tensor


In [ ]:
H = np.load(f"{DATA_PATH}/241022/systemMatrix/H_matrix_{H_SETTING}.npy")
H_tensor = matrix_to_tensor(H, 128, 128)

In [ ]:
# 時刻 t の行列 A(t)
A_t = csr_matrix(H_tensor[63, 63, :, :])

# 時刻 t+1 の行列 A(t+1)
A_t1 = csr_matrix(H_tensor[64, 64, :, :])

# 中間時刻 t + 0.5 の行列を推定
A_half = interpolate_sparse_matrices(A_t, A_t1)

# 結果の表示
def plot_heatmap(matrix, title):
    plt.figure(figsize=(4, 4))
    plt.imshow(matrix.toarray(), cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title(title)
    plt.xlabel('列')
    plt.ylabel('行')
    plt.show()

# 各時刻の行列をヒートマップで表示
plot_heatmap(A_t, "A(t)")
plot_heatmap(A_t1, "A(t+1)")
plot_heatmap(A_half, "A(t + 0.5)")

At=A_t.toarray()
